# Query atlas.brabant.nl

See https://atlas.brabant.nl/arcgis/rest/services for list of available datasources.

Arcgis Python API: https://developers.arcgis.com/python/

## Libraries

In [ ]:
import os
import requests
import pandas as pd
import geopandas as gpd
from arcgis.mapping import MapImageLayer
from arcgis.features import SpatialDataFrame
import matplotlib.pyplot as plt
import mplleaflet

%matplotlib inline

pd.set_option('max_columns', 999)
pd.set_option('max_rows', 999)

## Functions

In [ ]:
def get_mapserver_services(base_url = "https://atlas.brabant.nl/ArcGIS/rest/services"):
    """
    Retrieves a list of available ArcGIS services of type 'MapServer' for a given base_url.
    
    Arguments:
        base_url: ArcGIS base url (usually ending with 'services')
    
    Returns:
        List of mapserver names
    """
    
    url = "%s?f=pjson" % base_url
    
    # Retrieve service list in json, extract services
    r = requests.get(url)
    results = r.json()
    services_dict = results['services']
    
    # Select only services of type MapServer
    mapserver_names = [service['name'] for service in services_dict if service['type'] == 'MapServer']
    
    return mapserver_names



def get_all_layers(base_url = "https://atlas.brabant.nl/ArcGIS/rest/services"):
    """
    Retrieves overview of available MapServer services under 'base_url' including their layers.
    
    Arguments:
        base_url: ArcGIS base url (usually ending with 'services')
    
    Returns:
        DataFrame with columns 'service_name', 'layer_number', 'layer_name'
    """
    
    available_services = get_mapserver_services(base_url)
    
    df_layer_info = pd.DataFrame(columns=['service_name', 'layer_number', 'layer_name'])
    for service_name in available_services:
        print("Checking service: %s" % service_name)
        
        # Get layers and layer numbers        
        mapserver_url = "%s/%s/MapServer" % (base_url, service_name)
        layers = MapImageLayer(mapserver_url).layers

        for layer_number in range(len(layers)):
            try:
                layer_name = layers[layer_number].properties['name']
                df_layer_info = df_layer_info.append(
                    {'service_name': service_name, 'layer_number': int(layer_number), 'layer_name': layer_name}, 
                    ignore_index=True
                )
            except:
                print("Could not get layer info for layer %s of mapserver %s" % (layer_number, mapserver_url))
        
    return df_layer_info



def get_arcgis_data(base_url="https://atlas.brabant.nl/arcgis/rest/services",
                    service_name="stortplaatsen",
                    layer_numbers='All',
                    output_dir=None,
                    verbose=False):
    """
    Retrieves data from ArcGIS MapServer, selects layer and converts to 
    spatial dataframe. Optionally saves results as shapefiles.
    
    Arguments:
        base_url:      ArcGIS base url (usually ending with 'services')
        service_name:  Name of arcgis service provided (usually visible when accessing base_url in browser)
        layer_numbers: List with layer numbers to select. If 'All', all layers will be selected.
        output_dir:    Optional directory for results. Output shapefiles stored in subdir with name of service_name.
        verbose:       If True, show intermediate steps.
        
    Returns:
        Dict with layer names as keys and spatial dataframes with results as values.
    """
    
    mapserver_url = "%s/%s/MapServer" % (base_url, service_name)

    if verbose:
        print("Querying url: %s" % mapserver_url)
    map_image_layer = MapImageLayer(mapserver_url)
    
    if verbose:
        print("Available layers:")
        for layer in map_image_layer.layers:
            print("    %s" % layer.properties['name'])
    
    # Select layers
    layers = map_image_layer.layers
    if layer_numbers != 'All':
        layers = [layers[layer_number] for layer_number in layer_numbers]
    
    results = {}
    for layer in layers:
        
        # Create spatial dataframe from layer
        try:
            sdf = pd.DataFrame.spatial.from_layer(layer)
        except:
            print("Could not convert layer to spatial dataframe. Skipping..")
            continue
        
        # Save spatial dataframe to shapefile
        layer_name = layer.properties['name']
        if output_dir is not None:
            output_file_without_extension = "%s/%s/%s" % (output_dir, service_name, layer_name)
            output_file = "%s.shp" % output_file_without_extension
            if verbose:
                print("Saving layer to shapefile: %s" % output_file)
            
            try:
                sdf.spatial.to_featureclass(location=output_file)
            except:
                print("    Layer could not be saved. (Not convertible to shapefile?). Skipping..")
                # Remove files that failed to save correctly
                os.remove("%s.dbf" % output_file_without_extension)
                os.remove("%s.shp" % output_file_without_extension)
                os.remove("%s.shx" % output_file_without_extension)
        
        # Append spatial dataframe to dictionary
        results.update({layer_name: sdf})
    
    return results



def load_shapefile(shapefile, 
                      crs='epsg:4326', 
                      bounding_box={'xmin': 5.00, 'ymin': 51.3, 'xmax': 6.05, 'ymax': 51.85}):
    """
    Reads shapefile, converts to desired coordinate system and selects entries within bounding box
    
    Arguments:
        shapefile:     Path to input shapefile
        crs:           Coordinate reference system (default: 'epsg:4326', ie lat/lon).
                       Set to None to keep original crs.
        bounding_box:  Dict with bounding box area (default: {'xmin': 5.00, 'ymin': 51.3, 'xmax': 6.05, 'ymax': 51.85}).
                       Set to None to keep all entries.
    
    Returns:
        Geopandas GeoDataFrame
    """

    # Read shapefile
    gdf = gpd.read_file(shapefile)

    if crs != None:
        # Convert to desired coordinate reference system
        gdf = gdf.to_crs({'init': crs})

    if bounding_box != None:
        # Select entries within bounding box
        xmin, ymin, xmax, ymax = bounding_box['xmin'], bounding_box['ymin'], bounding_box['xmax'], bounding_box['ymax']
        gdf = gdf.cx[xmin:xmax, ymin:ymax]

    return gdf



def plot_leaflet(gdf, output_file=None):
    """
    Plot geodataframe on top of a leaflet background
    
    Arguments:
        gdf:          Geopandas GeoDataFrame
        output_file:  Path to output html file. If None, output is not saved to file.
    
    Returns:
        Leaflet plot
    """
    
    fig, ax = plt.subplots(figsize=(16,16), subplot_kw={'aspect':'equal'})
    plot = gdf.plot(color='red', edgecolor='red', alpha=1, ax=ax)
    leaflet = mplleaflet.display(fig=plot.figure)
    
    if output_file != None:
        # Export to file
        mplleaflet.show(fig=plot.figure, path=output_file)
    
    return leaflet

## Get list of available ArcGIS data services

In [ ]:
available_services = get_mapserver_services(base_url='https://atlas.brabant.nl/arcgis/rest/services')
available_services

In [ ]:
# This can take a minute or two
df_layer_info = get_all_layers(base_url='https://atlas.brabant.nl/arcgis/rest/services')

In [ ]:
df_layer_info

In [ ]:
df_layer_info.to_csv('../results/available_layers.csv')

## Retrieve data and store as shapefiles

### For single service name

In [ ]:
# arcgis_results = get_arcgis_data(base_url='https://atlas.brabant.nl/arcgis/rest/services',
#                                  service_name='aardkundige_waarden_v2',
#                                  layer_numbers='All',
#                                  output_dir='../results',
#                                  verbose=True)

### For multiple services and/or layers

In [ ]:
# Choose services and layers to extract -> add other ones!
relevant_services_layers = {'primair_meetnet': [0],
                            'geitenhouderijen_buffer': [0], 
                            'stortplaatsen': [4],
                            'Provinciale_wegen': [0],
                            'toestandsrapportage_natuur_service_alt': [20, 21],
                            'verkeersintensiteiten': [2],
                            'BWA_GeologieGeohydrologie': [0,2,3,6],
                            'aardkundige_waarden_v2': [0,1,2,3]}

# Loop over relevant services and layers. Store results as shapefiles in results dir.
for service_name, layer_numbers in relevant_services_layers.items():
    _ = get_arcgis_data(base_url='https://atlas.brabant.nl/arcgis/rest/services',
                                 service_name=service_name,
                                 layer_numbers=layer_numbers,
                                 output_dir='../results',
                                 verbose=True)

## Visualize using geopandas/leaflet

In [ ]:
gdf = load_shapefile(shapefile='../results/stortplaatsen/4 Voormalige stortplaatsen vlak.shp')
gdf.head(3)

In [ ]:
plot_leaflet(gdf, output_file='../results/map.html')